# Analysis 5

Importing packages

In [1]:
import pandas
from bokeh.io import show,output_notebook
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.sampledata.us_counties import data as counties

Root path

In [2]:
rootdir = "C:/Users/Ankit Mahajan/Documents/Github/mahajan_ankit/final project/"

Accepting user input

In [3]:
input_year = input('year = ')
states = [x.strip() for x in input('states = ').lower().split(",")]

year = 2009
states = CA,MA,NY,TX,MI,WA,WY,FL,IL


Retrieving processed Data

In [4]:
df = pandas.read_csv(rootdir+'/data/processed data/IRS_Data.csv', sep=",")
codata = pandas.read_csv(rootdir+'/data/raw data/co-ordinates/zip_codes_states.csv', sep=",")

Filtering data accordinig to user's input

In [5]:
df = df[df.year == int(input_year)]

Calculating unemployment claims for every zipcode

In [6]:
st = df
df = df.groupby(["zipcode"]).sum()

Calculating percentage of unemployment for the zip code

In [7]:
df["percentage"] = pandas.Series(data = df.n02300/df.n1*100, index=df.index)

Preparing data for visualisation

In [8]:
df["state"]= pandas.Series()
df.state = map(str, df.state)
for a in df.index:
    df["state"].set_value(a, st[st.zipcode==a]["state"].iloc[0])   
df["ZIP"] = df.index
codata = codata.rename(columns={'zip_code': 'ZIP'})
df = df.merge(codata ,on=["ZIP"], left_index=True)
df.ZIP = map(int, df.ZIP)

Calculating Unemployment rate

In [9]:
result = []
county_names = [county['name'] for county in counties.values() if county["state"] in states]


for a in county_names:
    temp = df[df.county == a]
    if temp.n1.sum() == 0:
        result.append(0)
    else:
        result.append(temp.n02300.sum()/temp.n1.sum()*100)

Visualizing data using a map

In [10]:
output_notebook()
palette.reverse()

county_xs = [county["lons"] for county in counties.values() if county["state"] in states]
county_ys = [county["lats"] for county in counties.values() if county["state"] in states]

color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=result,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="USA unemployement distribution according to county", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=1000
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="black", line_width=0.5)
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Unemployment rate)", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
]

show(p)

Loading BokehJS ...